# HPE Container Platform API series - Lab 1
## Getting starting with the HPE Container Platform REST API - Session Authentication


**Requirements:**
- HPE Container Platform deployment
- IP address or FQDN of the HPE Container Platform's controller host and Gateway host
- a tenant user account   

**Recommended workshop and blog:**
- Understanding API basics and the value they provide (session ID: HACK-HSW01V). 
- You can also check out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal.

**Utilities:**   
- Postman, cURL  
- Jupyter Notebook server with bash kernel installed

**Definitions:**
**Definitions:**
- *HPE Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Container Platform ideal for helping enterprise customers accelerate their application development and deployment on **containers**, on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Container Platform visit the [HPE DEV portal](https://developer.hpe.com) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent for example, an office location, a business unit, an organization, or a project. A tenant is allocated a quota of resources (CPU, GPU, memory, storage, and Kubernetes clusters resources) by the platform administrator. All the resources used by a tenant are not shared with other tenants. A tenant user is granted the role of member or admin for the tenant.


## The HPE Container Platform REST API

>Note: If you are not already familiar with REST API calls and Postman, I encourage you to check out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads and how to make API calls. 

All the HPE Container Platform REST API calls contain: 

**The HTTP method (Verb):**

* *GET* – Retrieve object instance properties
* *POST* – Create a new object instance
* *PUT/PATCH* – Modify object instance properties (two different ways)
* *DELETE* - Remove an object instance 

**The API endpoint URL:**

   https://API-endpoint/api/v2/[object]
   
   In our HPE CP deployment, the communication with HPE CP REST API endpoint is through the Gateway host on port 8080:
   
   https://[Gateway-IP-address-or-fqdn]:8080/api/v2/[object]
  

## Session Authentication in a multi-tenant environment

The HPE CP REST API allows you to execute multiple actions programmatically, from performing administrative tasks like creating Kubernetes clusters to deploying applications for various use cases in a shared multi-tenant environment.

All the REST API calls you can do against the HPE Container Platform API must be authenticated with a "token". HPE Container Platform uses a *‘session location’* as token. In a multi-tenant environment, you retrieve the session location by issuing an authentication request in the following form:
* A POST request against object **/api/v2/session**, with the username/password credentials and the Tenant name in the body.
* The session location is then used in the HTTP header of subsequent requests.
* For each subsequent call, you specify a new HTTP Header with its key set to **X-BDS-SESSION** and its value set to the session location value and used as the *working tenant context*.  

### Using Postman

Postman can be used to issue REST API calls to the HPE Container Platform. The following capture shows the information passed by Postman to the API. It is a **POST** operation against **/api/v2/session** to create a session object. The header specifies JSON as the exchange format and the body provides the user credentials and tenant name. The target HPE Container Platform endpoint (the gateway) IP address, the user credentials and tenant name are defined in a group of variables called a Postman *Environment*. These variables typically describe a given target environment and are expected to be set before the API calls. Postman also allows you to extract the session location from the response header and store it in a Postman environment variable for later use. This is accomplished by using a simple JavaScript code as shown in the *Tests* tab. 

<img src="Pictures/HPECP-Postman-post-session.gif" height="800" width="600" align="left">

Save the value of the session _Location_ from the response header above, to use it in all your subsequent API calls. For example, here we call the HPE Container Platform API to verify we can make REST API calls within the tenant working context by fetching information about the session just established.

<img src="Pictures/HPECP-Postman-Get-session.png" height="800" width="600" align="left">

<img src="Pictures/HPECP-Postman-Get-session-info-response.png" height="800" width="600" align="left">

Notice the session location will expire after 24 hours (1440 minutes). 
 
Let's do the equivalent API call to create the session using cURL. Information on cURL can be found [here](https://curl.haxx.se/)

### Using cURL

Postman lets you generate snippets of code in various languages that will help you translate Postman queries into your prefered code. You can use the **Code** link under the blue **Send** button to open the *GENERATE CODE SNIPPETS* and select your preferred language. As shown in the picture below, we use this feature to generate the cURL code for the POST query for the authentication session:


<img src="Pictures/Postman-GTW-Generate-code-snippet.png" height="800" width="600" align="left">

#### Initialize the environment.
   
Let's first define the environment variables according to your HPE Container Platform working context (credentials and tenant name), and the HPE Container Platform API system endpoint:

In [1]:
#
# environment variables to be adjusted/verified by the student
#
username="student1" # your HPE CP tenant login credentials - username
password="stuASP2020" # your HPE CP tenant login credentials - password
#
# fixed environment variables setup by the HPE CP Lab administrator
#
controller_endpoint="gateway1.hpedevlab.net:8080"
tenantname="K8sHackTenant"
echo "your working context is:" $username-$password-$tenantname 

your working context is: student1-stuASP2020-K8sHackTenant


#### -1- Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo $SessionId

This is your session location:  /api/v2/session/b5f7412e-fec3-46d9-8747-86196a91590d
b5f7412e-fec3-46d9-8747-86196a91590d


#### -2- Making subsequent API calls:
For each subsequent call, you must set a new HTTP Header with its key set to **X-BDS-SESSION** and its value set to the session location value. This session token is used as the *working tenant* context. Here you will fetch information about your session you have just established with HPE Container Platform as a tenant user.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/session"
    }
  },
  "session_expiry_minutes": 1440,
  "_embedded": {
    "sessions": [
      {
        "_links": {
          "self": {
            "href": "/api/v2/session/b5f7412e-fec3-46d9-8747-86196a91590d"
          },
          "all_sessions": {
            "href": "/api/v2/session"
          }
        },
        "user": "/api/v1/user/1155",
        "user_name": "student1",
        "tenant": "/api/v1/tenant/5",
        "tenant_name": "K8sHackTenant",
        "role": "/api/v1/role/3",
        "role_name": "Member",
        "expiry": "2020-5-18 19:12:44",
        "expiry_time": 1589821964,
        "is_site_admin_view": false,
        "is_cluster_superuser": false
      }
    ]
  }
}


### -3- Closing your login session:

Although session have a time to live (TTL) of 24 hours, it is **best practice** in REST API programming to cleanup and delete those sessions when done. You use a DELETE /api/v2/session/SessionId to achieve this.

In [4]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Sun, 17 May 2020 17:13:27 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session has been deleted.

## Summary

In this tutorial, you learned how to authenticate to the HPE Container Platform API endpoint and retrieve object data by providing the authentication session location in the header. 

Now, let's continue with our next lab:
* [Lab 2](2-WKSHP-HPECP-DeployApp-K8S-Tenant-tf.ipynb)